# <font color=yellow> Market Rates Integration </font>
This example demonstrates a streamlined process for integrating market rates into Kyriba, focusing on both interest and foreign exchange (FX) rates essential for treasury operations. The use case consists of two separate steps:

1. **Retrieve and Import SHIBOR Interest Rates**

  * Fetch SHIBOR (Shanghai Interbank Offered Rate) interest rate data for a specified date range from https://shibor.org.
  * Process and transform the data to a CSV to match Kyriba's import format.
  * Import the CSV into Kyriba.

2. **Set and Import FX Rate**

  * Manually input the quote currency (the base currency is EUR - system reference currency of the API Playground), spot date, bid and ask rates.
  * Transform the input data into a structured format suitable for Kyriba.
  * Import the data into Kyriba.

## <font color='yellow'>Step 0.1: Code for token generation and API management</font>

In [ ]:
#@title Select **servicePack** the Sample should run
from subprocess import getstatusoutput
servicePack = "DEMO"

if servicePack == 'DEMO':
  branch = 'main'
else:
  branch = servicePack
print(branch)
rm = getstatusoutput("rm Colab_shared_code_for_API_usage -rf ")
clone = getstatusoutput("git clone -l -s --branch " +  branch + " https://github.com/kyriba/Colab_shared_code_for_API_usage.git")
import importlib
try:
  importlib.reload(sample_requests)
except:
  print()
from Colab_shared_code_for_API_usage import sample_requests

In [ ]:
#@title Import custom modules

import time
import matplotlib
from pprint import pprint
import matplotlib.pyplot as plot
from datetime import datetime,date
import random


## <font color='yellow'>Step 0.2: Connect and  retrieve token</font>

In [ ]:
#@title <font color='lime'>Connect</font>
print('token:')
token = sample_requests.login()

## <font color='yellow'>Step 1: Retrieve and Import SHIBOR Interest Rates</font>
Retrieve SHIBOR interest rates from https://shibor.org, transform the data into the format expected by Kyriba, and import it.

In [ ]:
#@title <font color='lime'>Step 1.1: Get SHIBOR Interest Rates by period</font>
start_date = "2024-02-01" #@param {type:"date"}
end_date = "2024-02-02" #@param {type:"date"}
show_rates = True #@param {type:"boolean"}

import requests
import pandas as pd
from datetime import datetime

csv_filename = f"shibor_rates_{start_date}_to_{end_date}.csv"
def fetch_and_process_shibor_data(start_date, end_date):
    # Construct the URL with query parameters for the GET request
    url = f"https://www.shibor.org/ags/ms/cm-u-bk-shibor/ShiborHis?lang=en&startDate={start_date}&endDate={end_date}"

    response = requests.get(url)

    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Extract the 'records' part of the data
        records = data.get('records', [])

        # Process each record to format data as required
        processed_records = []
        for record in records:
            business_date = datetime.strptime(record["showDateEN"], "%d %b %Y").strftime("%m-%d-%Y")
            for key, value in record.items():
                if key not in ["showDateEN", "showDateCN"]:
                    product_code = f"SHIBOR{key}"
                    rate = value
                    processed_records.append({
                        "businessDate": business_date,
                        "rate": rate,
                        "productCode": product_code,
                        "transactionTime": "",
                        "productDescription": "",
                        "securityId": ""
                    })

        # Convert processed records into a DataFrame
        global df
        df = pd.DataFrame(processed_records)

        # Reorder the DataFrame here if needed
        df = df[["businessDate", "transactionTime", "rate", "productCode", "securityId", "productDescription"]]

        # Save the DataFrame to a CSV file
        csv_filename = f"shibor_rates_{start_date}_to_{end_date}.csv"
        df.to_csv(csv_filename, index=False)

        print(f"Data successfully processed and saved to {csv_filename}")
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

if end_date >= start_date:
    fetch_and_process_shibor_data(start_date, end_date)

    if show_rates:
        print()
        print(df.to_markdown())
else:
    print('\033[31mStart date cannot be later than the End date.')
    print('Select the correct dates and try again.')


In [ ]:
#@title <font color='lime'>Step 1.2: Import Interest Rates to Kyriba</font>
usePayload = True #@param {type:"boolean"}
traces = True #@param {type:"boolean"}

with open(csv_filename, 'rb') as csv_file:
    binary_file_data = csv_file.read()


result = sample_requests.import_data(token, binary_file_data, csv_filename, 'CME_MD', usePayload, traces)

print(result)
print()
print(f"Data {csv_filename} uploaded into Kyriba.")

## <font color='yellow'>Step 2: Set and Import FX Rate</font>
Import the FX rate set in the form.

In [ ]:
#@title <font color='lime'>Step 2.1: Enter FX Rate</font>
currency = "CAD" #@param ["USD", "GBP", "CAD"]
spotDate = "2021-10-21" #@param {type:"date"}
bid =  1.13#@param {type:"number"}
ask = 1.14 #@param {type:"number"}
usePayload = True #@param {type:"boolean"}
traces = True #@param {type:"boolean"}

transformedDate =   spotDate[5:7] + spotDate[8:10]  + spotDate [0:4]

data = ("FX" + transformedDate + "MANUAL    "+currency+"                 "+str(bid)+"              "+str(ask)+"\n")

today = date.today()
now = datetime.now()
dt_string = str(today) + 'T' + now.strftime("%H:%M")
filename = dt_string + "-" + currency + ".csv"
if traces:
  print('Data: ' + data)
  if usePayload:
    print('Data will be sent by payload.\n')
  else:
    print('Data will be sent by file:' + filename + '.\n')

res = sample_requests.import_data(token, data, filename,'API_IMP_MD', usePayload, traces)